# TV Script Generation 电视剧本生成

In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).

在这个项目中，您将使用RNN生成您自己的[Simpsons](https://en.wikipedia.org/wiki/The_Simpsons)电视剧。 您将使用27个季节的[Simpsons数据集](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) 部分脚本。 您将构建的神经网络将为[Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern) 的场景生成一个新的电视剧。

## Get the Data 获取数据

The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

数据已经为您提供。 您将使用原始数据集的子集。 它只由Moe's Cavern的场景组成。 这不包括其他版本的酒馆，如“Moe's Cavern”，“Flaming Moe's”，“Moe's Family Feed-Bag”等。

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]
print(text[:500])

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simp


## Explore the Data 探索数据
Play around with `view_sentence_range` to view different parts of the data.

玩 `view_sentence_range` 来查看数据的不同部分。

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435


In [8]:
sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

Number of lines: 4257
Average number of words in each line: 11.50434578341555


In [13]:
view_sentence_range = (0, 10)
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

view_sentence_range = (648, 658)
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


The sentences 648 to 658:
Barney_Gumble: If you were such a big shot, why aren't you still making movies?
Barney_Gumble: Moe?... Moe?
Moe_Szyslak: Oh no! My favorite aggie!
Moe_Szyslak: You stole my bit! You... That's my bit ya... Ya... ooh, ooh, you stole my bit!
Director: Cut!


## Implement Preprocessing Functions 实现预处理功能


The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

任何数据集的首要任务是预处理。 实现以下预处理功能：
- 查找表
- 标记标点符号

### Lookup Table 查找表
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

要创建一个单词嵌入，您首先需要将单词转换为ids。 在此功能中，创建两个字典：
- 从词到字典的字典，我们称之为 `vocab_to_int`
- 从id到word的字典，我们将调用 `int_to_vocab`

在以下元组 `（vocab_to_int，int_to_vocab）` 中返回这些字典

In [16]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word:ii for ii, word in enumerate(vocab, 1)}
    int_to_vocab = {ii:word for ii, word in enumerate(vocab, 1)}
    return vocab_to_int, int_to_vocab

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation  标记标点符号
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

我们将使用空格作为分隔符将脚本分割成一个单词数组。然而，像句点和感叹号这样的标点使得神经网络难以区分"bye"和"bye!".一词。

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:

实现函数 `token_lookup` 来返回一个将用于标记符号的dict，如“！”进入“|| Exclamation_Mark ||”。为符号为关键字，值为令牌的以下符号创建字典：

- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

该字典将用于标记符号，并在其周围添加分隔符( space)。这将符号与自己的单词分开，使得神经网络更容易预测下一个单词。确保您不要使用可能会被误解为一个字的令牌。而不是使用令牌“破折号”，请尝试使用像“||dash||”这样的东西。

In [25]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    return {".":"||period||", ",":"||comma||",  "\"":"||quotation||", ";":"||semicolon||", 
            "!":"||exclamation||",  "?":"||question||",  "(":"||left_parentheses||",  ")":"||right_parentheses||", "--":"dash", "\n":"||return" }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it 预处理所有数据并保存
Running the code cell below will preprocess all the data and save it to file.

运行下面的代码单元将预处理所有的数据并保存到文件。

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)
print("preprocess data saved")

preprocess data saved


# Check Point 检查点
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

这是你的第一个检查点。 如果你决定回到笔记本电脑或者重新启动笔记本电脑，你可以从这里开始。 预处理数据已保存到磁盘。

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
print("preprocess data loaded")

preprocess data loaded


## Build the Neural Network 建立神经网络

You'll build the components necessary to build a RNN by implementing the following functions below:

您将通过实现以下功能构建构建RNN所需的组件：

- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU  检查TensorFlow版本和访问GPU

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input 输入
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:

实现`get_inputs()`函数来创建神经网络的TF占位符。 应该创建以下占位符：

- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

- 使用[TF占位符](https://www.tensorflow.org/api_docs/python/tf/placeholder)`name`参数输入名为“输入”的文本占位符。
- 目标占位符
- 学习率占位符

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

在以下元组`(Input, Targets, LearningRate)`中返回占位符

In [4]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize  构建RNN单元并初始化

Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
- Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

在[`MultiRNNCell`](https://www.tensorflow.org/)中堆叠一个或多个[`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell)。
- Rnn大小应该使用`rnn_size`设置
- 使用MultiRNNCell [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state)函数启动单元格状态
- 使用[`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)将名称“initial_state”应用于初始状态

在下面的元组`(Cell, InitialState)`中返回单元格和初始状态

In [5]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, "initial_state")
        
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding 单词嵌入
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

使用TensorFlow将嵌入应用于`input_data`。 返回嵌入序列。

In [6]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN 建立RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

您在`get_init_cell()`函数中创建了一个RNN单元格。 使用单元格创建RNN的时间。
- 使用[`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)构建RNN
- 使用[`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)将名称“final_state”应用于最终状态

在下面的元组`(Outputs, FinalState)`中返回output和final_state状态

In [7]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, "final_state")
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network 建立神经网络
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

将上述实现的功能应用于：
- 使用`get_embed(input_data，vocab_size，embed_dim)`函数将嵌入应用于`input_data`。
- 使用`cell`和你的`build_rnn(cell，inputs)`函数构建RNN。
- 使用线性激活和`vocab_size`作为输出数量应用完全连接的层。

在以下元组中返回逻辑和最终状态 `(Logits，FinalState)`

In [37]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data,  vocab_size, embed_dim)
    outputs,  final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=tf.nn.relu)

    return logits, final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches 分批
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

实现 `get_batches` 来创建输入和结果的分批使用`int_text`,分批应该是一个Numpy数组，形状为`(number of batches, 2, batch size, sequence length)`。每批包含两个元素：
- 第一个元素是一批**输入**，形状为`[batch size, sequence length]`
- 第二个元素是一批**目标**，形状为`[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

如果您无法使用足够的数据填写最后一个批次，请删除最后一批。

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:

例如，`get_batches（[1，2，3，4，5，6，7，8，9，10，11，12，13，14，15]，2，3）`将返回一个Numpy数组如下：

```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [74]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequenc
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    count = len(int_text)//(batch_size * seq_length)
    x = np.array(int_text[: batch_size*seq_length*count])
    y = np.array(int_text[1: batch_size*seq_length*count+1])
    x = x.reshape(-1, 1, batch_size, seq_length)
    y = y.reshape(-1, 1, batch_size, seq_length)    
    ret = np.concatenate((x, y), axis=1)
    return ret

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training 训练神经网络
### Hyperparameters 超参数

Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

调整以下参数：

- 将`num_epochs`设置为历元数。
- 将`batch_size`设置为批量大小。
- 将`rnn_size`设置为RNN的大小。
- 将`embed_dim`设置为嵌入的大小。
- 将`seq_length`设置为序列长度。
- 将`learning_rate`设置为学习率。
- 将`show_every_n_batches` 设置为神经网络应该打印进度的批次数。

In [97]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 5
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 100

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph 构建图
Build the graph using the neural network you implemented.

使用您实现的神经网络构建图。

In [98]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
print("done")

done


## Train 训练
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

在预处理数据上训练神经网络。 如果您很难受损，请查看[forms](https://discussions.udacity.com/)以查看是否有任何人遇到同样的问题。

In [99]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/107   train_loss = 8.822
Epoch   0 Batch  100/107   train_loss = 6.145
Epoch   1 Batch   93/107   train_loss = 5.819
Epoch   2 Batch   86/107   train_loss = 5.706
Epoch   3 Batch   79/107   train_loss = 5.885
Epoch   4 Batch   72/107   train_loss = 5.283
Epoch   5 Batch   65/107   train_loss = 4.970
Epoch   6 Batch   58/107   train_loss = 5.226
Epoch   7 Batch   51/107   train_loss = 5.138
Epoch   8 Batch   44/107   train_loss = 4.573
Epoch   9 Batch   37/107   train_loss = 4.657
Epoch  10 Batch   30/107   train_loss = 4.874
Epoch  11 Batch   23/107   train_loss = 4.801
Epoch  12 Batch   16/107   train_loss = 4.512
Epoch  13 Batch    9/107   train_loss = 4.283
Epoch  14 Batch    2/107   train_loss = 4.530
Epoch  14 Batch  102/107   train_loss = 4.756
Epoch  15 Batch   95/107   train_loss = 4.595
Epoch  16 Batch   88/107   train_loss = 4.736
Epoch  17 Batch   81/107   train_loss = 4.572
Epoch  18 Batch   74/107   train_loss = 4.549
Epoch  19 Batch   67/107   train_l

## Save Parameters 保存参数
Save `seq_length` and `save_dir` for generating a new TV script.

保存`seq_length`和`save_dir`用于生成新的电视剧脚本。

In [100]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))
print("params saved")

params saved


# Checkpoint 检查点

In [101]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()
print("params loaded")

params loaded


## Implement Generate Functions 实现生成函数
### Get Tensors 获取Tensors

Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:

使用函数[`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name)从`loaded_graph`获取张量。 使用以下名称获取张量：

- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

返回下面的元组中的张量`（InputTensor，InitialStateTensor，FinalStateTensor，ProbsTensor）`

In [105]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    probs_tensor = loaded_graph.get_tensor_by_name("probs:0")
    return input_tensor, initial_state_tensor, final_state_tensor, probs_tensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word 选择单词
Implement the `pick_word()` function to select the next word using `probabilities`.

实现`pick_word()`函数，使用`probabilities`来选择下一个单词。

In [106]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    idx = np.argmax(probabilities)
    return int_to_vocab[idx]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script 生成电视剧脚本
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

这将为您生成电视剧本。 将`gen_length`设置为要生成的电视剧本的长度。

In [109]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(to barney) hey moe, hey, hey, two, everybody, who's all over the, but you're really comin'.
moe_szyslak:(to self) hey, get a beer, the little...
barney_gumble:(high voice, do this friend!
bart_simpson:(to camera) oh man, this is gonna have great. and you could do say to marge.
moe_szyslak:(nervous) oh, me?
bart_simpson:(singing) when you're in a lot of
moe_szyslak:(to homer) hey, i love you, but i am gonna have a beer.
barney_gumble: okay, okay!
moe_szyslak: yeah, that ain't never!
moe_szyslak:(big, guys, it is all right.
moe_szyslak:(nervous) oh, me?
bart_simpson:(singing) when you're in a lot of
moe_szyslak:(to homer) hey, i love you, but i am gonna have a beer.
barney_gumble: okay, okay!
moe_szyslak: yeah, that ain't never!
moe_szyslak:(big,


# The TV Script is Nonsensical 电视剧脚本是荒谬的
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.

没关系，电视剧没有任何意义。 我们训练了不到一兆字节的文字。 为了获得良好的效果，您必须使用较小的词汇量或获得更多数据。 幸运的是有更多的数据！ 正如我们在乞讨这个项目中提到的，这是[另一个数据集](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data)的一个子集。 我们没有你训练所有的数据，因为这将花费太长时间。 然而，您可以自由地对所有数据进行神经网络训练。 


# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.